# Retrieve と RetrieveAndGenerate の違い

Knowledge bases for Amazon Bedrock を利用した検索には 2 種類の API が提供されています。

- [RetrieveAndGenerate](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html)
- [Retrieve](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_Retrieve.html)

文字通り Retrieve API はベクトルデータベースから情報の取得のみを行い、 RetrieveAndGenerate API はベクトルデータベースから情報の取得した後、回答生成まで行います。

WIP

<img src="../images/01/01.png">

RetrieveAndGenerate API と Retrieve API の挙動を確認してみます。

## RetrieveAndGenerate API

In [ ]:
import boto3

KNOWLEDGEBASE_ID = "" # ナレッジベース ID を記載
REGION = "us-west-2"

model_arn = "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
knowledge_base = boto3.client("bedrock-agent-runtime")

knowledge_base.retrieve_and_generate(
    input={"text": "私はエンジニアです。オンボーディング初日に必要なことを教えてください。"},
    retrieveAndGenerateConfiguration={
        "type": 'KNOWLEDGE_BASE',
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn
        },
    },
)